In [2]:
"""import tensorflow as tf
from tensorflow.nn import relu, dropout

class GCN(tf.keras.Model):
  def __init__(self, in_features, hidden_features, dropout_rate):
    super(GCN, self).__init__()

    self.fc1 = tf.keras.layers.Dense(hidden_features, activation=relu)
    self.fc2 = tf.keras.layers.Dense(1, activation="sigmoid")  # Sigmoid for link probability
    self.dropout = dropout(dropout_rate)

  def call(self, inputs):
    x1, x2, adj = inputs  # source features, destination features, adjacency matrix

    # Combine source and destination features
    node_features = tf.concat([x1, x2], axis=1)

    # ... (message passing with adj as before)

    # Output layer for link prediction
    output = self.dropout(self.fc2(hidden))
    return output

# Example usage
model = GCN(16, 8, 0.5)  # in_features, hidden_features, dropout

# ... (Load features, adjacency matrix, prepare positive/negative samples)

# Train/Validation split (using your preferred splitting method)
train_features_source, train_features_dest, train_adj, train_labels = ...
val_features_source, val_features_dest, val_adj, val_labels = ...
test_features_source, test_features_dest, test_adj = ...

# Compile model for training
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Training loop (using train/val data)
# ...

# Testing loop (using test data)
model.evaluate((test_features_source, test_features_dest, test_adj), test_labels)

# Prediction on new unseen edges
new_source_features, new_dest_features = ...  # Features for new potential links
predictions = model((new_source_features, new_dest_features, test_adj))
"""

'import tensorflow as tf\nfrom tensorflow.nn import relu, dropout\n\nclass GCN(tf.keras.Model):\n  def __init__(self, in_features, hidden_features, dropout_rate):\n    super(GCN, self).__init__()\n\n    self.fc1 = tf.keras.layers.Dense(hidden_features, activation=relu)\n    self.fc2 = tf.keras.layers.Dense(1, activation="sigmoid")  # Sigmoid for link probability\n    self.dropout = dropout(dropout_rate)\n\n  def call(self, inputs):\n    x1, x2, adj = inputs  # source features, destination features, adjacency matrix\n\n    # Combine source and destination features\n    node_features = tf.concat([x1, x2], axis=1)\n\n    # ... (message passing with adj as before)\n\n    # Output layer for link prediction\n    output = self.dropout(self.fc2(hidden))\n    return output\n\n# Example usage\nmodel = GCN(16, 8, 0.5)  # in_features, hidden_features, dropout\n\n# ... (Load features, adjacency matrix, prepare positive/negative samples)\n\n# Train/Validation split (using your preferred splitting me

In [3]:
"""import pandas as pd
import tensorflow as tf
from tensorflow.nn import relu, dropout

# Function to read CSV and create data structures
def process_data(csv_file):
  df = pd.read_csv(csv_file)
  embedding_dict = {}
  adj_matrix = tf.zeros((max_node_id + 1, max_node_id + 1))  # Assuming node IDs start from 0

  for index, row in df.iterrows():
    node_id = row["node_id"]  # Replace with your actual column name for node ID
    embedding = row[embedding_feature_columns]  # Replace with column names for embedding
    embedding_dict[node_id] = embedding

    # Process connections (assuming separate columns for source and destination)
    source = row["source_node_id"]  # Replace with your actual column name
    dest = row["destination_node_id"]  # Replace with your actual column name
    adj_matrix[source, dest] = 1  # Update adjacency matrix for connections

  return embedding_dict, adj_matrix

# ... (Load your CSV file)

embedding_dict, adj_matrix = process_data(csv_file)

# Function to prepare training and testing data (replace with your logic for negative sampling)
def prepare_data(embedding_dict, adj_matrix, test_ratio=0.2):
  # Extract positive samples from connection information
  positive_samples = []
  for source, row in embedding_dict.items():
    for dest in row.keys():  # Assuming connections are stored in a dictionary within embeddings
      if source != dest:
        positive_samples.append((source, dest))

  # Generate negative samples (replace with your preferred negative sampling method)
  negative_samples = [(source, random.choice(list(embedding_dict.keys()))) for source, _ in positive_samples]

  # Split data into training and testing sets
  train_size = int(len(positive_samples) * (1 - test_ratio))
  train_data = positive_samples[:train_size] + negative_samples[:train_size]
  test_data = positive_samples[train_size:] + negative_samples[train_size:]

  return train_data, test_data

# ... (Prepare training and testing data using your logic for negative sampling)"""

'import pandas as pd\nimport tensorflow as tf\nfrom tensorflow.nn import relu, dropout\n\n# Function to read CSV and create data structures\ndef process_data(csv_file):\n  df = pd.read_csv(csv_file)\n  embedding_dict = {}\n  adj_matrix = tf.zeros((max_node_id + 1, max_node_id + 1))  # Assuming node IDs start from 0\n\n  for index, row in df.iterrows():\n    node_id = row["node_id"]  # Replace with your actual column name for node ID\n    embedding = row[embedding_feature_columns]  # Replace with column names for embedding\n    embedding_dict[node_id] = embedding\n\n    # Process connections (assuming separate columns for source and destination)\n    source = row["source_node_id"]  # Replace with your actual column name\n    dest = row["destination_node_id"]  # Replace with your actual column name\n    adj_matrix[source, dest] = 1  # Update adjacency matrix for connections\n\n  return embedding_dict, adj_matrix\n\n# ... (Load your CSV file)\n\nembedding_dict, adj_matrix = process_data(c

In [36]:
url = "bolt://localhost:7687"
user = "neo4j"
password = "neo4jkngbq"

from py2neo import Graph

# Connect to Neo4j
graph = Graph(url, auth=(user, password))

# Query to get node embeddings and relationships
query = """
MATCH (n:SubNode{filename:"software"})-[r]->(m:Interface)
RETURN id(n) AS source_id, id(m) AS target_id, n.embeddings AS src_embedding, m.embeddings AS tgt_embedding
"""

data = graph.run(query).data()

In [37]:
def convert_embedding_str_to_list(embedding_str):
    emb_split = embedding_str.split(',')
    emb_list = [float(x) for x in emb_split]
    return emb_list

node_embeddings = {}
for d in data:
    src_id = d['source_id']
    tgt_id = d['target_id']
    if src_id not in node_embeddings:
        node_embeddings[src_id] = convert_embedding_str_to_list(d['src_embedding'])
    if tgt_id not in node_embeddings:
        node_embeddings[tgt_id] = convert_embedding_str_to_list(d['tgt_embedding'])
        
node_id_mapping = {node_id: idx for idx, node_id in enumerate(node_embeddings.keys())}

In [38]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([
    [node_id_mapping[d['source_id']], node_id_mapping[d['target_id']]]
    for d in data
], dtype=torch.long).t().contiguous()


In [39]:
node_features = torch.tensor([node_embeddings[node_id] for node_id in sorted(node_embeddings.keys())], dtype=torch.float)


In [40]:
from torch_geometric.data import Data

# Creating the PyG data object
graph_data = Data(x=node_features, edge_index=edge_index)

In [48]:
graph_data

Data(x=[424, 1536], edge_index=[2, 1182])

In [66]:
from torch_geometric.data import DataLoader
from torch_geometric.transforms import RandomLinkSplit

# Split edges and prepare data loaders
transform_split = RandomLinkSplit(is_undirected=True, add_negative_train_samples=True, num_val=0.1, num_test=0.1)
train_data, val_data, test_data = transform_split(graph_data)

# Assuming you need to manually handle subgraph extraction if required:
# You would typically prepare subgraphs manually or use a custom dataset class here
# For now, let's proceed without the subgraph extraction to focus on other parts of the setup

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)


In [67]:
test_data

Data(x=[424, 1536], edge_index=[2, 168], edge_label=[18], edge_label_index=[2, 18])

In [68]:
val_data

Data(x=[424, 1536], edge_index=[2, 150], edge_label=[18], edge_label_index=[2, 18])

In [69]:
train_data

Data(x=[424, 1536], edge_index=[2, 150], edge_label=[150], edge_label_index=[2, 150])

In [73]:
len(test_data.x[1])

1536

In [82]:
train_data

Data(x=[424, 1536], edge_index=[2, 150], edge_label=[150], edge_label_index=[2, 150])

In [91]:
train_loader.__dict__

{'follow_batch': None,
 'exclude_keys': None,
 'dataset': Data(x=[424, 1536], edge_index=[2, 150], edge_label=[150], edge_label_index=[2, 150]),
 'num_workers': 0,
 'prefetch_factor': None,
 'pin_memory': False,
 'pin_memory_device': '',
 'timeout': 0,
 'worker_init_fn': None,
 '_DataLoader__multiprocessing_context': None,
 '_dataset_kind': 0,
 'batch_size': 32,
 'drop_last': False,
 'sampler': <torch.utils.data.sampler.RandomSampler at 0x338de0950>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x338de6fd0>,
 'generator': None,
 'collate_fn': <torch_geometric.loader.dataloader.Collater at 0x338cdd410>,
 'persistent_workers': False,
 '_DataLoader__initialized': True,
 '_IterableDataset_len_called': None,
 '_iterator': None}

Data(x=[424, 1536], edge_index=[2, 150], edge_label=[150], edge_label_index=[2, 150])

In [75]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class SEALModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(SEALModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels//2)
        self.conv3 = GCNConv(hidden_channels//2, 1)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x.view(-1)

model = SEALModel(len(test_data.x[1]), 256)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train ():
    model.train()

In [79]:
def train():
    model.train()
    total_loss = 0
    for data in test_data:
        print(data)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        label = data.y.float()
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# Train the model
for epoch in range(50):
    loss = train()
    print(f'Epoch {epoch+1}: Loss: {loss:.4f}')

('x', tensor([[-0.0223, -0.0170,  0.0130,  ..., -0.0061,  0.0018, -0.0123],
        [-0.0250, -0.0242,  0.0024,  ..., -0.0373, -0.0035,  0.0080],
        [-0.0214, -0.0113,  0.0196,  ..., -0.0205, -0.0139,  0.0093],
        ...,
        [-0.0230, -0.0203,  0.0094,  ..., -0.0122, -0.0089, -0.0315],
        [ 0.0021, -0.0144,  0.0180,  ..., -0.0115, -0.0083, -0.0157],
        [ 0.0118, -0.0150, -0.0151,  ...,  0.0089, -0.0108, -0.0319]]))


AttributeError: 'tuple' object has no attribute 'x'

In [78]:
train_loader